In [1]:
import os
import json
import requests

from tqdm import tqdm

from utils import async_get, get, get_client_access_token

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import numpy as np

In [2]:
client_id = "d71970014054446a9f65903c9fd8afdd"
client_secret = "2be6c841192c4d4b9eda0e7faf2354c3"

access_token = get_client_access_token(client_id, client_secret)
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

playlists_api_endpoint = "https://api.spotify.com/v1/users/thesoundsofspotify/playlists"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
num_playlists = get(requests.Session(), 
                    "https://api.spotify.com/v1/users/thesoundsofspotify/playlists", "", 
                    authorization_header, True)['total']

https://api.spotify.com/v1/users/thesoundsofspotify/playlists                1.68s


In [4]:
queries = []
num_items = num_playlists
limit = 50
for i in range(int(num_items / limit) + 1):
    queries.append("?offset={}&limit={}".format(i * limit , limit))

results = await async_get(playlists_api_endpoint, queries, headers=authorization_header, verbose=True)

File                                   Completed at
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=250&limit=50                2.90s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=300&limit=50                2.93s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=150&limit=50                2.95s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=350&limit=50                2.96s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=100&limit=50                2.97s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=0&limit=50                3.05s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=450&limit=50                3.06s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=50&limit=50                3.10s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=550&limit=50                4.12s
https://api.spotify.com/

https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4050&limit=50               13.77s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4100&limit=50               14.03s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4150&limit=50               14.32s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4200&limit=50               14.50s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4250&limit=50               14.60s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4300&limit=50               14.75s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4350&limit=50               14.80s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4400&limit=50               14.83s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=4450&limit=50               14.87s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?of

https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=7800&limit=50               24.35shttps://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8050&limit=50               24.35s

https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8100&limit=50               24.66s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8150&limit=50               24.84s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8200&limit=50               24.91s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8250&limit=50               25.34s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8300&limit=50               25.53s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8400&limit=50               25.57s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?offset=8350&limit=50               25.59s
https://api.spotify.com/v1/users/thesoundsofspotify/playlists?of

In [5]:
playlists = sum([result['items'] for result in results], [])

In [6]:
def scrape_sounds(playlists, playlist_filter, playlist_processor, 
                  early_stop=lambda x : False, agg_results=lambda x : x):        
    """
    # Function to scrape The Sounds of Spotify playlists
    # playlist_filter - a function that takes in a playlist and returns True or False, indiciating whether to process that playlist
    # playlist_processor - a function that processes a playlist and returns one or more values that will be returned by the scraper
    # early_stop - a function that takes in a set of playlists and returns True or False, indicating if the scraper should stop scraping more playlists
    # agg_results - a function that takes the results produced by playlist_processor and and aggregates it in some way to be returned in aggregated form by this scraper
    """
    processor_results = []
    for playlist in playlists:
        if playlist_filter(playlist):
            processor_results.append(playlist_processor(playlist))

        if early_stop(playlists):
            break

    return agg_results(processor_results)

In [7]:
countries = json.loads(open("countries.json", "r").read())

In [8]:
def the_sound_of_filter(playlist):
    split_result = playlist['name'].replace('/','-').replace('-', ' ').split()
    is_sound_of = " ".join(split_result[:3]) == "The Sound of"
    return is_sound_of

def the_sound_of_process(playlist):
    split_result = playlist['name'].replace('/','-').replace('-', ' ').split()
    rest_of_string = " ".join(split_result[3:])
    not_city = len(split_result[-1]) != 2
    city = not not_city
    not_country = rest_of_string not in countries
    country = not not_country
    
    genre = not_city and not_country
    
    if genre:
        genre_name = rest_of_string
        return "genres", genre_name, playlist
    elif city:
        city_location = rest_of_string
        return "cities", city_location, playlist
    elif country:
        country = rest_of_string
        return "countries", country, playlist

def the_sound_of_early_stop(playlists):
    return any(["Aveiro" in playlist['name'] for playlist in playlists])
    
def the_sound_of_aggregate(results):
    data_dict = { category : [] for category in ['genres', 'cities', 'countries'] }
    for result in results:
        data_dict[result[0]].append({ "playlist" : result[2], 
                                      "name" : result[1]
                                    })
        
    return data_dict

In [9]:
playlist_dict = scrape_sounds(playlists, the_sound_of_filter, 
                              the_sound_of_process, agg_results=the_sound_of_aggregate)

In [10]:
# From the features, extract just the ones we want
features_to_get = ['energy', 'liveness', 'speechiness', 'acousticness', 'instrumentalness', 
                   'danceability', 'loudness', 'valence', 'tempo']
feature_names = features_to_get + ["popularity"]

def extract_features(features):
    # If a song hasn't been featurized, return an empty list
    if features is None:
        return []
    else:
        return [features[feature_to_get] for feature_to_get in features_to_get]

# Get all of the features we are interested in from a set of track URIs
def get_features_for_songs(songs, chunk_size=100):
    all_features = []
    num_chunks = int(len(songs) / chunk_size)
    remainder = int(((len(songs) / chunk_size) % 1) * chunk_size)
    for i in range(num_chunks):
        low = chunk_size * i
        high = chunk_size * (i + 1)
        uris = songs[low:high]
        
        chunk_track_features = sp.audio_features(uris)      
        all_features += [extract_features(features) for features in chunk_track_features]
        
    # Finish the last chunk
    if remainder != 0:
        try:
            low = high
            high = high + remainder
        except:
            low = 0
            high = remainder
        uris = songs[low:high]

        chunk_track_features = sp.audio_features(uris)
        all_features += [extract_features(features) for features in chunk_track_features]
    
    return all_features


# Get all of the features we are interested in from the tracks in a playlist
def get_features_for_playlist(playlist):
    tracks_to_get = []

    tracks = sp.user_playlist_tracks("thesoundsofspotify", playlist['uri'])
    tracks_to_get += [track for track in tracks['items'] if track['track'] is not None]

    while tracks['next']:
        tracks = sp.next(tracks)
        tracks_to_get += [track for track in tracks['items'] if track['track'] is not None]

    tracks = tracks_to_get
    
    def get_uris(tracks):
        uris = []
        for track in tracks:
            try:
                uri = str(track['track']['uri'])
            except:
                continue
            uris.append(uri)
        return uris
    
    def get_popularities(tracks):
        popularities = []
        for track in tracks:
            try:
                pop = track['track']['popularity']
            except:
                continue
            popularities.append(pop)
        return popularities
    
    track_uris = get_uris(tracks)    
    
    tracks_features = get_features_for_songs(track_uris)
    
    track_names = []
    popularities = []
    ret_uris = []
    tracks_features_good = []
    # Some songs don't have features
    for track, uri, features in zip(tracks, track_uris, tracks_features):
        if len(features) != 0:
            track_names.append(track['track']['name'])
            popularities.append(track['track']['popularity'])
            ret_uris.append(uri)
            tracks_features_good.append(features)
            
    ret_features = [features + [pop] for pop, features in zip(popularities, tracks_features_good)]
    
    return track_names, ret_uris, ret_features

In [11]:
for category_name, (playlist_type, playlist_data) in zip(['genre', 'country', 'city'], 
                                                          playlist_dict.items()):
    print("Playlist Type: ", playlist_type)

    all_data_list = []
    average_features_list = []
    all_songs_list = []
    
    average_features_filename = "{0}/data_{0}_average_features.json".format(playlist_type)
    all_songs_filename = "{0}/data_{0}_songs.json".format(playlist_type)
    all_data_filename = "{0}/data_{0}.json".format(playlist_type)
    
    # Skip this playlist type if we've already generated all of the data
    if all([os.path.exists(filename) for filename in [average_features_filename, all_songs_filename, all_data_filename]]):
        continue
    
    for playlist_item in tqdm(playlist_data):           
        name = playlist_item['name']
        # Remove non-ascii characters
        name = name.encode("ascii", errors="ignore").decode()
        
        songs_filename = "{0}/songs/data_{0}_{1}_songs.json".format(playlist_type, name)
        
        playlist = playlist_item['playlist']
        
        names, uris, features = get_features_for_playlist(playlist)
        
        average_features = np.mean(np.array(features), axis=0)
        
        all_data_dict = {}
        all_data_dict['name'] = name
        
        for feature_name, average_feature in zip(feature_names, average_features):
            all_data_dict[feature_name] = average_feature

        all_data_dict['uri'] = playlist['uri']
        
        # Just the features
        features_dict = all_data_dict.copy()
        average_features_list.append(features_dict)
        
        # Song information
        songs_list = []
        for song_name, song_uri, song_features in zip(names, uris, features):
            songs_dict = {}
            songs_dict['name'] = song_name
            songs_dict['uri'] = song_uri
            for feature_name, song_feature in zip(feature_names, song_features):
                songs_dict[feature_name] = song_feature
            
            songs_list.append(songs_dict)
        
        all_songs_list.append({ 'name' : name, 
                                'songs' : songs_list})
        
        # Append song info to all data
        all_data_dict['songs'] = songs_list
        all_data_list.append(all_data_dict)
        
        open(songs_filename, "w").write(json.dumps(songs_list, indent=4))
        
    open(average_features_filename, "w").write(json.dumps(average_features_list, indent=4))
    open(all_songs_filename, "w").write(json.dumps(all_songs_list, indent=4))
    open(all_data_filename, "w").write(json.dumps(all_data_list, indent=4))

  0%|          | 0/4524 [00:00<?, ?it/s]

Playlist Type:  genres
Playlist Type:  cities
{'grant_type': 'client_credentials'}
https://accounts.spotify.com/api/token
{'access_token': 'BQBjWjSZWdR1FhjTEa0q-Jo_9GUVALZqrKVVg6kiaxm_tGTNo-iVWD7sGDInuLzF7JFMiDw8PeC4Mi-lso4', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


  0%|          | 0/173 [00:00<?, ?it/s]

Playlist Type:  countries


100%|██████████| 173/173 [00:51<00:00,  3.34it/s]
